# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

ROOT_PATH = '/Users/moshfiqur/Learning/machine-learning/object-detection-playground'
DATA_PATH = os.path.join(ROOT_PATH, 'data')
TF_MODELS_PATH = '/Users/moshfiqur/Learning/machine-learning/tf-models/models'
TEST_IMAGES_PATH = '/Users/moshfiqur/Learning/machine-learning/object-detection-playground/images'

# This is needed since the notebook is stored in the object_detection folder.
# sys.path.append(os.path.join(TF_MODELS_PATH, 'research'))
sys.path.append(os.path.join(TF_MODELS_PATH, 'research', 'object_detection'))

print(sys.path)

from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
    raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

print(tf.VERSION)


/Users/moshfiqur/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/moshfiqur/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['', '/Users/moshfiqur/Learning/machine-learning/object-detection-playground/ssd_mobilenet_v2_coco_2018_03_29', '/Users/moshfiqur/Learning/machine-learning/tf-models/models/research', '/Users/moshfiqur/Learning/machine-learning/tf-models/models/research/slim', '/Users/moshfiqur/anaconda3/lib/python36.zip', '/Users/moshfiqur/anaconda3/lib/python3.6', '/Users/moshfiqur/anaconda3/lib/python3.6/lib-dynload', '/Users/moshfiqur/.local/lib/python3.6/site-packages', '/Users/moshfiqur/anaconda3/lib/python3.6/site-packages', '/Users/moshfiqur/anaconda3/lib/python3.6/site-packages/aeosa', '/Users/moshfiqur/anaconda3/lib/python3.6/site-packages/object_detection-0.1-py3.6.egg', '/Users/moshfiqur/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/Users/moshfiqur/.ipython', '/Users/moshfiqur/Learning/machine-learning/tf-models/models/research/object_detection']
1.4.0


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

/Users/moshfiqur/Learning/machine-learning/tf-models/models/research/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/moshfiqur/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/moshfiqur/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/moshfiqur/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/moshfiqur/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/moshfiqur/anaconda3/lib/python

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [12]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v2_coco_2018_03_29'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(DATA_PATH, 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

In [13]:
if not os.path.isdir(MODEL_NAME):
    opener = urllib.request.URLopener()
    opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    tar_file = tarfile.open(MODEL_FILE)
    for file in tar_file.getmembers():
        file_name = os.path.basename(file.name)
        if 'frozen_inference_graph.pb' in file_name:
            tar_file.extract(file, os.getcwd())
else:
    print('Model already exists!')

Model already exists!


## Load a (frozen) Tensorflow model into memory.

In [14]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [15]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

print(categories)

[{'id': 1, 'name': 'person'}, {'id': 2, 'name': 'bicycle'}, {'id': 3, 'name': 'car'}, {'id': 4, 'name': 'motorcycle'}, {'id': 5, 'name': 'airplane'}, {'id': 6, 'name': 'bus'}, {'id': 7, 'name': 'train'}, {'id': 8, 'name': 'truck'}, {'id': 9, 'name': 'boat'}, {'id': 10, 'name': 'traffic light'}, {'id': 11, 'name': 'fire hydrant'}, {'id': 13, 'name': 'stop sign'}, {'id': 14, 'name': 'parking meter'}, {'id': 15, 'name': 'bench'}, {'id': 16, 'name': 'bird'}, {'id': 17, 'name': 'cat'}, {'id': 18, 'name': 'dog'}, {'id': 19, 'name': 'horse'}, {'id': 20, 'name': 'sheep'}, {'id': 21, 'name': 'cow'}, {'id': 22, 'name': 'elephant'}, {'id': 23, 'name': 'bear'}, {'id': 24, 'name': 'zebra'}, {'id': 25, 'name': 'giraffe'}, {'id': 27, 'name': 'backpack'}, {'id': 28, 'name': 'umbrella'}, {'id': 31, 'name': 'handbag'}, {'id': 32, 'name': 'tie'}, {'id': 33, 'name': 'suitcase'}, {'id': 34, 'name': 'frisbee'}, {'id': 35, 'name': 'skis'}, {'id': 36, 'name': 'snowboard'}, {'id': 37, 'name': 'sports ball'}, {

## Helper code

In [16]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [19]:
# TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
test_images = os.listdir(TEST_IMAGES_PATH)
print(test_images)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

['.DS_Store', '0.jpg', 'bsp_mobile_gallery_335_2679503276.jpg', 'bsp_mobile_gallery_335_2679652596.jpg', 'bsp_mobile_gallery_335_2679771534.jpg', 'bsp_mobile_gallery_335_2679804100.jpg', 'bsp_mobile_gallery_335_2679810482.jpg', 'bsp_mobile_gallery_335_2679811841.jpg', 'bsp_mobile_gallery_335_2680226091.jpg', 'bsp_mobile_gallery_335_2680412215.jpg', 'bsp_mobile_gallery_335_2863174585.png', 'bsp_mobile_gallery_335_2863306903.jpg', 'bsp_mobile_gallery_335_2863322245.jpg', 'bsp_mobile_gallery_335_2863322524.jpg', 'bsp_mobile_gallery_335_3165867588.jpg', 'bsp_mobile_gallery_335_3263910195.jpg', 'bsp_mobile_gallery_335_3263963941.jpg', 'bsp_mobile_gallery_335_3263966124.jpg', 'bsp_mobile_gallery_335_3263988392.jpg', 'bsp_mobile_gallery_335_3264017714.jpg', 'bsp_mobile_gallery_335_3264019323.jpg', 'bsp_mobile_gallery_335_3345799056.jpg', 'bsp_mobile_gallery_335_3432645671.jpg', 'bsp_mobile_gallery_335_3486551657.jpg', 'bsp_mobile_gallery_335_3591676897.jpg']


In [20]:
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in ['num_detections', 'detection_boxes', 'detection_scores', 'detection_classes', 'detection_masks']:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
            
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
            
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
            
            # Run the inference
            output_dict = sess.run(tensor_dict, feed_dict={image_tensor: np.expand_dims(image, 0)})
            
            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

### Predict and Add BBOX on Images

In [21]:
for file_name in test_images:
    if '.jpg' not in file_name:
        continue
    
    image_path = os.path.join(TEST_IMAGES_PATH, file_name)
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # print(output_dict)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)


InvalidArgumentError: NodeDef mentions attr 'identical_element_shapes' not in Op<name=TensorArrayV3; signature=size:int32 -> handle:resource, flow:float; attr=dtype:type; attr=element_shape:shape,default=<unknown>; attr=dynamic_size:bool,default=false; attr=clear_after_read:bool,default=true; attr=tensor_array_name:string,default=""; is_stateful=true>; NodeDef: Preprocessor/map/TensorArray = TensorArrayV3[clear_after_read=true, dtype=DT_FLOAT, dynamic_size=false, element_shape=<unknown>, identical_element_shapes=true, tensor_array_name="", _device="/job:localhost/replica:0/task:0/device:CPU:0"](Preprocessor/map/TensorArrayUnstack/strided_slice). (Check whether your GraphDef-interpreting binary is up to date with your GraphDef-generating binary.).
	 [[Node: Preprocessor/map/TensorArray = TensorArrayV3[clear_after_read=true, dtype=DT_FLOAT, dynamic_size=false, element_shape=<unknown>, identical_element_shapes=true, tensor_array_name="", _device="/job:localhost/replica:0/task:0/device:CPU

### Predict and Print the Scores

In [22]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        for file_name in test_images:
            if '.jpg' not in file_name:
                continue
            
            image_path = os.path.join(TEST_IMAGES_PATH, file_name)
            
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            # Actual detection.
            (result_boxes, result_scores, result_classes, result_num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})

            print(result_scores)
            # Here output the category as string and score to terminal
            # print([category_index.get(i) for i in classes[0]])
            # print(scores)
            # print(scores)
            print()
            # break


InvalidArgumentError: NodeDef mentions attr 'identical_element_shapes' not in Op<name=TensorArrayV3; signature=size:int32 -> handle:resource, flow:float; attr=dtype:type; attr=element_shape:shape,default=<unknown>; attr=dynamic_size:bool,default=false; attr=clear_after_read:bool,default=true; attr=tensor_array_name:string,default=""; is_stateful=true>; NodeDef: Preprocessor/map/TensorArray_2 = TensorArrayV3[clear_after_read=true, dtype=DT_INT32, dynamic_size=false, element_shape=<unknown>, identical_element_shapes=true, tensor_array_name="", _device="/job:localhost/replica:0/task:0/device:CPU:0"](Preprocessor/map/TensorArrayUnstack/strided_slice). (Check whether your GraphDef-interpreting binary is up to date with your GraphDef-generating binary.).
	 [[Node: Preprocessor/map/TensorArray_2 = TensorArrayV3[clear_after_read=true, dtype=DT_INT32, dynamic_size=false, element_shape=<unknown>, identical_element_shapes=true, tensor_array_name="", _device="/job:localhost/replica:0/task:0/device

### Predict and Print the Labels only upto certain threshold along with predicted probablity and BBOX

In [18]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        for file_name in test_images:
            if '.jpg' not in file_name:
                continue
            
            image_path = os.path.join(TEST_IMAGES_PATH, file_name)
            
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            # Actual detection.
            (result_boxes, result_scores, result_classes, result_num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})

            result_boxes = np.squeeze(result_boxes)
            result_classes = np.squeeze(result_classes)
            result_scores = np.squeeze(result_scores)
            
            print(file_name)
            print('*'*5)
            for index, score in enumerate(result_scores):
                if score < 0.5:
                    continue
                
                label = category_index[result_classes[index]]['name']
                ymin, xmin, ymax, xmax = result_boxes[index]
                
                print(label, score, ymin, xmin, ymax, xmax)
                print()
                print()


0.jpg
*****
person 0.93631625 0.033080608 0.007139474 0.9911357 0.9917121


bsp_mobile_gallery_335_2679503276.jpg
*****
person 0.9039627 0.0 0.32678312 0.990568 0.6885193


bsp_mobile_gallery_335_2679652596.jpg
*****
bsp_mobile_gallery_335_2679771534.jpg
*****
spoon 0.73928756 0.22962809 0.19106743 0.87040377 0.93266594


spoon 0.5193809 0.35962185 0.36367866 0.8804091 0.9549725


bsp_mobile_gallery_335_2679804100.jpg
*****
person 0.82360935 0.0036728382 0.03943196 0.99824595 0.9950862


bsp_mobile_gallery_335_2679810482.jpg
*****
cup 0.76730484 0.18761241 0.13682169 0.6951729 0.34312576


bottle 0.56232 0.16248119 0.6982514 0.8145172 0.91024953


bsp_mobile_gallery_335_2679811841.jpg
*****
bsp_mobile_gallery_335_2680226091.jpg
*****
bsp_mobile_gallery_335_2680412215.jpg
*****
person 0.8259676 0.006612599 0.086120725 0.99424964 0.97542846


bsp_mobile_gallery_335_2863306903.jpg
*****
person 0.90617865 0.0016807318 0.17068142 0.99453235 0.8815482


bsp_mobile_gallery_335_2863322245.jpg
